In [5]:
from bs4 import BeautifulSoup

In [6]:
from tdk.core import TurkishWord

In [7]:
import trstop

In [8]:
import requests

In [9]:
from selenium import webdriver

In [10]:
import nltk

In [11]:
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [12]:
import time

In [13]:
def jobUrls_kariyer(jobTitle):
    url = "https://www.kariyer.net/is-ilanlari?kw=" + jobTitle
    driver = webdriver.Firefox(executable_path = r'C:\Users\The Creator\Desktop\ResEnh\geckodriver.exe')
    driver.get(url)
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    jListings = soup.find_all("a", class_ = "k-ad-card rounded")
    jListingsRefs = []
    for ref in jListings:
        jListingsRefs.append("https://www.kariyer.net/" + ref["href"])
    return jListingsRefs

In [14]:
#job listings urls from the 1st page
jListings = jobUrls_kariyer("veri analisti")
time.sleep(2)
jobDescriptions = []
for url in jListings:
    jPage = requests.get(url)
    jSoup = BeautifulSoup(jPage.content, "html.parser")
    jDescription = jSoup.find_all("div" , attrs={"class" : "genel-nitelikler" })
    for jD in jDescription:
        jobDescriptions.append(jD.text)
    time.sleep(2)

In [16]:
jListings[:10]

['https://www.kariyer.net//is-ilani/keyvan-havacilik-anonim-sirketi-veri-analisti-3126648',
 'https://www.kariyer.net//is-ilani/enuygun-com-veri-analisti-3329700',
 'https://www.kariyer.net//is-ilani/kalyon-yatirim-holding-a-s-veri-analisti-3301144',
 'https://www.kariyer.net//is-ilani/avansas-veri-analisti-3109452',
 'https://www.kariyer.net//is-ilani/odas-elektrik-uretim-sanayi-ve-ticaret-a-s-veri-analisti-3329669',
 'https://www.kariyer.net//is-ilani/yucelen-grup-veri-analisti-3287537',
 'https://www.kariyer.net//is-ilani/uyguner-deri-pazarlama-ve-tic-a-s-veri-analisti-3337145',
 'https://www.kariyer.net//is-ilani/sabanci-universitesi-is-ve-veri-analisti-3314216',
 'https://www.kariyer.net//is-ilani/metlife-emeklilik-ve-hayat-a-s-bes-gozetim-veri-analisti-3301488',
 'https://www.kariyer.net//is-ilani/zeplin-turizm-tasimacilik-yatirim-ltd-sti-veri-analisti-ve-crm-uzmani-3294770']

In [17]:
jobDescriptions[:10]

['\n\r\nGENEL NİTELİKLER VE İŞ TANIMI                                            \n* Dünya çapındaki havacılık verilerini analiz etme ve dijital formata dönüştürme,* Farklı kaynaklardan gelen (CD, Online Data, Basılı Materyal vb.) ve resmi olarak onaylanmış verilerden belirlenen standartlar doğrultusunda veri tabanı oluşturma.*Teknik ve uygulama sorunlarını analiz etme ve gerektiğinde çözüm odaklı çalışabilme.* Departman hedeflerine ulaşmak için ekiple etkin iletişim ve iş birliği. ',
 "\n\r\nGENEL NİTELİKLER VE İŞ TANIMI                                            \nWingie Enuygun Group, ayda 20 milyon ziyaretçisi ve 70 milyon sayfa gösterimi ile Türkiye’nin lider seyahat pazaryeridir. Wingie Enuygun Group ekibi olarak ülkemizde uçak bileti, otel, kurumsal seyahat, otobüs bileti, araç kiralama alanlarında kullanıcılarımıza mükemmel servisi vermek için çalışıyoruz.En önemlisi, her iş günü sonunda milyonlarca Enuygun ve Wingie kullanıcısı için değer katan işler yapmış olmanın gururuyla e

In [18]:
def removeDigits(word):
    letters = 'abcdefghijklmnopqrstuvwxyzğıöşçüâ'
    for letter in word:
        if letter not in letters:
            word = word.replace(letter,'')
    return(word)

In [19]:
#pre-processing
njd = []
oneTxt = ''
stopWords = stopwords.words('turkish')
nstopWords = ['veveya','vb','vs','yıl','e','bana','']
enstopWords = stopwords.words('english')
stopWords.extend(nstopWords)
stopWords.extend(enstopWords)

for description in jobDescriptions:
    description = description.replace('GENEL NİTELİKLER VE İŞ TANIMI', '')
    description = description.replace('QUALIFICATIONS AND JOB DESCRIPTION', '')
    oneTxt = oneTxt + ' ' + description
    
oneTxt = oneTxt.strip()
symbols = ',/:()[]{}#*-+.&;'

for char in symbols:
    if char in oneTxt:
        oneTxt = oneTxt.replace(char,' ')
        
tokens = oneTxt.split() 

for token in tokens:
    token = token.lower() #lowercase
    token = removeDigits(token) #digits,spaces and special characters
    token = token.strip()
    if token != '' and token not in stopWords:
        #token = token.split()
        njd.append(token) 

In [20]:
njd

['dünya',
 'çapındaki',
 'havacılık',
 'verilerini',
 'analiz',
 'etme',
 'dijital',
 'formata',
 'dönüştürme',
 'farklı',
 'kaynaklardan',
 'gelen',
 'cd',
 'online',
 'data',
 'basılı',
 'materyal',
 'resmi',
 'olarak',
 'onaylanmış',
 'verilerden',
 'belirlenen',
 'standartlar',
 'doğrultusunda',
 'veri',
 'tabanı',
 'oluşturma',
 'teknik',
 'uygulama',
 'sorunlarını',
 'analiz',
 'etme',
 'gerektiğinde',
 'çözüm',
 'odaklı',
 'çalışabilme',
 'departman',
 'hedeflerine',
 'ulaşmak',
 'ekiple',
 'etkin',
 'iletişim',
 'iş',
 'birliği',
 'wingie',
 'enuygun',
 'group',
 'ayda',
 'milyon',
 'ziyaretçisi',
 'milyon',
 'sayfa',
 'gösterimi',
 'türkiyenin',
 'lider',
 'seyahat',
 'pazaryeridir',
 'wingie',
 'enuygun',
 'group',
 'ekibi',
 'olarak',
 'ülkemizde',
 'uçak',
 'bileti',
 'otel',
 'kurumsal',
 'seyahat',
 'otobüs',
 'bileti',
 'araç',
 'kiralama',
 'alanlarında',
 'kullanıcılarımıza',
 'mükemmel',
 'servisi',
 'vermek',
 'çalışıyoruz',
 'önemlisi',
 'iş',
 'günü',
 'sonunda',
 

In [ ]:
#n-grams

In [38]:
#lemmatizer

In [21]:
stopWords

['acaba',
 'ama',
 'aslında',
 'az',
 'bazı',
 'belki',
 'biri',
 'birkaç',
 'birşey',
 'biz',
 'bu',
 'çok',
 'çünkü',
 'da',
 'daha',
 'de',
 'defa',
 'diye',
 'eğer',
 'en',
 'gibi',
 'hem',
 'hep',
 'hepsi',
 'her',
 'hiç',
 'için',
 'ile',
 'ise',
 'kez',
 'ki',
 'kim',
 'mı',
 'mu',
 'mü',
 'nasıl',
 'ne',
 'neden',
 'nerde',
 'nerede',
 'nereye',
 'niçin',
 'niye',
 'o',
 'sanki',
 'şey',
 'siz',
 'şu',
 'tüm',
 've',
 'veya',
 'ya',
 'yani',
 'veveya',
 'vb',
 'vs',
 'yıl',
 'e',
 'bana',
 '',
 'i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'ha